# City and Weather API's

## City API

In [71]:
import pandas as pd
import os
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak'
path = local_project_path+'\\my_booking\\my_booking'
print(path)
os.chdir(path)
chemin_fichier_csv = "./city.csv"
df = pd.read_csv(chemin_fichier_csv, header=None, names=['city','nan'])
cities = df['city'].str.replace(' ', '-').tolist()

# list(cities)

C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\my_booking\my_booking


In [72]:
import requests

api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:

    payload = {
        'city' : city,
        'limit' : 1,
        'country' : 'france',
        'country-code' : 'fr',
        'format' : 'json'
        }
    
    r_city = requests.get(api_url, 
                          params=payload)

    enriched_cities.append({'city': city,
                            'lat' : float(r_city.json()[0]['lat']),
                            'lon' : float(r_city.json()[0]['lon'])})
    
# display(enriched_cities)


In [73]:
df = pd.DataFrame.from_dict(enriched_cities)

In [74]:
import pandas as pd
import plotly.express as px
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        mapbox_style='carto-positron',
                        hover_name='city',
                        height=600)

fig.show()

## Weather API

In [75]:
os.chdir('../../')

with open("API_key.txt", "r") as file:
    API_key = str(file.readline())

exclude = 'current,minutely,hourly,alerts'
weather_cities = []

for city in enriched_cities:
    lat = city['lat']
    lon = city['lon']
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&exclude={exclude}&appid={API_key}&units=metric'
    weather_raw = requests.get(url).json()
    weather_cities.append(weather_raw)

# display(weather_cities)

In [76]:
pd.json_normalize(weather_cities).head()

,cod,message,cnt,list,city.id,city.name,city.coord.lat,city.coord.lon,city.country,city.population,city.timezone,city.sunrise,city.sunset
0,200,0,40,"[{'dt': 1711994400, 'main': {'temp': 12.08, 'f...",6435453,Huisnes-sur-Mer,48.6360,-1.5115,FR,0,7200,1711950186,1711996572
1,200,0,40,"[{'dt': 1711994400, 'main': {'temp': 12.02, 'f...",2978640,St-Malo,48.6495,-2.0260,FR,50676,7200,1711950308,1711996696
2,200,0,40,"[{'dt': 1711994400, 'main': {'temp': 11.97, 'f...",3034483,Bayeux,49.2765,-0.7025,FR,15963,7200,1711949958,1711996411
3,200,0,40,"[{'dt': 1711994400, 'main': {'temp': 11.14, 'f...",3003796,Le Havre,49.4939,0.1080,FR,185972,7200,1711949752,1711996228
4,200,0,40,"[{'dt': 1711994400, 'main': {'temp': 12.99, 'f...",2982652,Rouen,49.4405,1.0940,FR,112787,7200,1711949519,1711995988


In [87]:
# pd.json_normalize(weather_cities)['city.name']
# pd.json_normalize(weather_cities)['list']
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])['weather'][0]

In [90]:
weather_datas = pd.DataFrame()

for id, city in enumerate(pd.json_normalize(weather_cities)['city.name']):

    df3 = pd.json_normalize(pd.json_normalize(weather_cities)['list'][id])
    df3['city'] = city
    df3['lat'] = pd.json_normalize(weather_cities)["city.coord.lat"][id]
    df3['lon'] = pd.json_normalize(weather_cities)["city.coord.lon"][id]
    
    weather_datas = pd.concat([weather_datas, df3])

weather_datas['dt_txt'] = pd.to_datetime(weather_datas['dt_txt']) 
display(weather_datas.head(3))

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,rain.3h,sys.pod,city,lat,lon
0,1711994400,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000.0,1.0,2024-04-01 18:00:00,12.08,11.43,10.33,12.08,997,...,1.75,52,8.23,224,13.42,1.81,d,Huisnes-sur-Mer,48.636,-1.5115
1,1712005200,"[{'id': 803, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-01 21:00:00,9.94,6.52,8.44,9.94,1000,...,1.50,77,8.28,219,13.90,NaN,n,Huisnes-sur-Mer,48.636,-1.5115
2,1712016000,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000.0,0.0,2024-04-02 00:00:00,9.61,5.98,9.61,9.61,1004,...,0.00,95,8.73,232,13.52,NaN,n,Huisnes-sur-Mer,48.636,-1.5115


In [91]:
col_drop = ['dt', 'weather', 'visibility', 'main.temp', 'main.temp_min', 'main.temp_max', 'main.pressure','main.sea_level',
            'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all', 'wind.deg', 'wind.gust', 'rain.3h']

weather_datas.drop(columns=col_drop, inplace=True)


In [94]:
weather_during_day = weather_datas[weather_datas['sys.pod']=='d'].drop(columns='sys.pod')

In [95]:
weather_day_mean = weather_during_day.groupby('city').mean().reset_index()

In [100]:
display(weather_day_mean[['city','pop']].sort_values('pop').head())
display(weather_day_mean[['city','main.feels_like']].sort_values('main.feels_like',ascending=False).head())
display(weather_day_mean[['city','wind.speed']].sort_values('wind.speed').head())

,city,pop
1,Aix-en-Provence,0.0132
6,Bauduen,0.0144
13,Collioure,0.0144
0,Aigues-Mortes,0.0160
29,Saintes-Maries-de-la-Mer,0.0160


,city,main.feels_like
13,Collioure,16.0088
21,Le Limpou,16.0080
31,Tarascon-sur-Ariège,15.6304
9,Biarritz,15.4736
5,Avignon,15.2968


,city,wind.speed
16,Grenoble,2.0704
3,Annecy,2.2432
31,Tarascon-sur-Ariège,2.3172
6,Bauduen,3.2356
21,Le Limpou,3.3172


Create a SCORE to select the city

In [101]:
weights = {'pop': -0.4, 'main.feels_like': 0.5, 'wind.speed': -0.1}

weather_day_mean['score'] = (weather_day_mean['pop'] * weights['pop'] +
                             weather_day_mean['main.feels_like'] * weights['main.feels_like'] +
                             weather_day_mean['wind.speed'] * weights['wind.speed'])

min_score = weather_day_mean['score'].min()
max_score = weather_day_mean['score'].max()
weather_day_mean['rate'] = 10 * (weather_day_mean['score'] - min_score) / (max_score - min_score)

display(weather_day_mean.sort_values('rate',ascending=False).head())

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score,rate
21,Le Limpou,0.0840,2024-04-04 07:12:00,16.0080,3.3172,43.4945,-1.4737,7.63868,10.000000
13,Collioure,0.0144,2024-04-04 07:12:00,16.0088,4.0016,42.5251,3.0832,7.59848,9.885189
31,Tarascon-sur-Ariège,0.0560,2024-04-04 07:12:00,15.6304,2.3172,42.8458,1.6068,7.56108,9.778374
9,Biarritz,0.0820,2024-04-04 07:12:00,15.4736,3.7620,43.4833,-1.5593,7.32780,9.112127
5,Avignon,0.0296,2024-04-04 07:12:00,15.2968,3.4832,43.9492,4.8059,7.28824,8.999143


In [155]:
fig = px.scatter_mapbox(weather_day_mean,
                        lat='lat',
                        lon='lon',
                        color='rate',
                        size='rate',
                        center=dict(lat=47, lon=2.2),
                        zoom=4.5,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "rate"],
                        height=600,
                        width=600*16/9)

fig.show()

In [154]:
top5_cities = weather_day_mean.sort_values('rate',ascending=False).head()
top5_cities['city']

21              Le Limpou
13              Collioure
31    Tarascon-sur-Ariège
9                Biarritz
5                 Avignon
Name: city, dtype: object

In [150]:

fig = px.scatter_mapbox(top5_cities,
                        lat='lat',
                        lon='lon',
                        color='rate',
                        size='rate',
                        center=dict(lat=top5_cities['lat'].mean(), lon=top5_cities['lon'].mean()),
                        zoom=6,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        text='city',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "rate"],
                        height=600,
                        width=600*16/9)


fig.show()